<a href="https://colab.research.google.com/github/IgorWounds/Stocktwits-AMC-Algotrading101/blob/main/Stocktwits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtain data from Stocktwits

In [1]:
import requests, json
import pandas as pd

df = pd.DataFrame(columns=['body', 'sentiment'])
response = requests.get('https://api.stocktwits.com/api/2/streams/symbol/AMC.json?limit=50')
data = json.loads(response.text)

for tweet in data['messages']:
  df = df.append(
      {'body': tweet['body'],
       'sentiment':tweet["entities"]["sentiment"]["basic"] if tweet["entities"]["sentiment"] else None
      }, 
      ignore_index=True
      )

df.head()

,body,sentiment
0,$AMC ken griffin should be put in prison \nYou...,Bullish
1,$AMC \nANY BEARS WANNA FIGHT!??,None
2,$AMC Apes have ransacked their pavilion and th...,Bullish
3,"$AMC raised my donation another 5,400 shares t...",Bullish
4,$AMC Holding 700+ woth 2/3rds been held for ov...,None


# Buy AMC when sentiment is Bullish and vice-versa

In [ ]:
!pip3 install alpaca-trade-api

In [ ]:
import requests, json
import time
import pandas as pd
import alpaca_trade_api as tradeapi

api_key = 'PKF27QM7BIGZXZYJ9XFN'
api_secret = 'EMdrfA0SjhBgrowy2xIBC0qvKUb1heooVJ8QBwYH'
base_url = 'https://paper-api.alpaca.markets'

api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

max_position_allowed = 100

def order(qty: float, side: str):
  api.submit_order(
      symbol='AMC', 
      qty=qty, 
      side=side, 
      time_in_force='gtc', 
      type='market', 
    )

while True:
  # Obtain current trading position
  current_position  = float(api.get_position("AMC").qty_available)

  df = pd.DataFrame(columns=['sentiment'])
  response = requests.get('https://api.stocktwits.com/api/2/streams/symbol/AMC.json?limit=30')
  data = json.loads(response.text)

  for tweet in data['messages']:
    df = df.append(
        {
        'sentiment':tweet["entities"]["sentiment"]["basic"] if tweet["entities"]["sentiment"] else None
        }, 
        ignore_index=True
        )
    
  df.dropna(inplace=True)
  df.replace(['Bullish', 'Bearish'], [1, 0], inplace=True)

  sentiment = df.sum()

  if int(sentiment) > len(df)/2 and current_position < max_position_allowed:
    try:
      order(10, 'buy')
    except Exception:
      print(f'Buy order failed: {Exception}')
  elif int(sentiment) < len(df)/2 and current_position > -max_position_allowed:
    try:
      order(10, 'sell')
    except Exception:
      print(f'Sell order failed: {Exception}')
  else:
    # Wait it out
    continue

  time.sleep(15*60)

KeyboardInterrupt: ignored